# Automatiseringsgrad Alderpensjon

### Oppskrift for å hente datafil
- Åpne NAV applikasjoner > Datavarehus
- Velg Saker og oppgaver > Sak Pesys > Kravbehandlingsprosess
- Åpne "PSAK120 Selvbetjenings- og automatiseringsgrad førstegangssaker og inntektsendringer. Tidsserie måned"
- Eksporter rapporter. Velg kun Totalrapport, filtype CSV-arkiv.
- Fjern de øverste radene i CSV-filen slik at øverste rad er kolonnenavn.
- Last opp filen til jupyter, for eksempel via onedrive (https://navno-my.sharepoint.com/)

In [ ]:
import pandas as pd
import operator
import numpy as np

In [ ]:
# Read file
# home directory might change
df_file = pd.read_csv("pensjon-data-analyse/data/Totalrapport_02232022.csv").drop('Unnamed: 8', axis=1)

In [ ]:
# Antall saker per stønadsområde
df_file.Stønadsområde.value_counts()

# Forbered data

In [ ]:
def filter_df(df, col_name: str, relate, field: str):
    return df[relate(df[col_name], field)]

In [ ]:
print(df_file.shape)
# Fjern saker opprettet av batch
df_no_batch = filter_df(df_file, "Opprettet av", operator.ne, "Batch")
print(df_no_batch.shape)
# Velg kun førstegangsbehandling
df_forstegang = filter_df(df_no_batch, "Sakstype Topp", operator.eq, "Førstegangsbehandling")
print(df_forstegang.shape)
# Alderspensjon
ytelse = "Alderspensjon"
df_alderspensjon = filter_df(df_forstegang, "Stønadsområde", operator.eq, ytelse)
print(df_alderspensjon.shape)

#Sett typen av "Antall innkomne saker" til numerisk
df_alderspensjon.loc[:,"Antall innkomne saker"] = df_alderspensjon["Antall innkomne saker"].apply(pd.to_numeric)

In [ ]:
# Grupper på interessante variabler
df_ap_grouped = df_alderspensjon.groupby(["År-måned", "Selvbetjening", "Automatiseringsgrad", "Utenlandstilsnitt"], as_index=False) 

In [ ]:
df_ap = df_ap_grouped["Antall innkomne saker"].sum().sort_values(["År-måned", "Selvbetjening", "Automatiseringsgrad", "Utenlandstilsnitt"]).reset_index(drop=True)

In [ ]:
df_ap_grouped["Antall innkomne saker"].sum()

In [ ]:
# Funksjon som regner ut totalt antall saker per dag og andel per gruppering
def add_total_and_andel(df):
    df = pd.merge(left=df, 
                  right=df.groupby("År-måned", as_index=False)["Antall innkomne saker"].sum(),
                  how="inner", 
                  on="År-måned", 
                  suffixes=("", " totalt"))
    df["Andel"] = df["Antall innkomne saker"] / df["Antall innkomne saker totalt"]
    return df

In [ ]:
df_ap = add_total_and_andel(df_ap)

In [ ]:
df_ap_tot = df_ap.groupby("År-måned", as_index=False)["Antall innkomne saker"].sum()

# Utgangspunkt Alderspensjon

In [ ]:
df_ap.head(12)

In [ ]:
df_ap_tot

# Plotting

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Filtre som funker på df_ap
_selvbetjent = df_ap["Selvbetjening"] == "Selvbetjent"
_innland = df_ap["Utenlandstilsnitt"] == "Nasjonal"
_auto = df_ap["Automatiseringsgrad"] == "Automatisk"

In [ ]:
# Angi farger for ulik grad av automatisering
automatiserings_colors = px.colors.qualitative.Set1[2::-1]

In [ ]:
def plot_automatisering1(df, box_color=px.colors.qualitative.Pastel[0]):
    '''
    df: en verdi per måned og automatiseringsgrad
    box_color: Farge på boxplot
    fig: Box for antall saker, line for automatiseringsgrad
    '''
    df_tot = df.groupby("År-måned", as_index=False)["Antall innkomne saker"].sum()
    
    _automatisk = df["Automatiseringsgrad"] == "Automatisk"
    _manuell = df["Automatiseringsgrad"] == "Manuell"
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    fig.add_trace(go.Bar(x=df_tot["År-måned"], y=df_tot["Antall innkomne saker"], 
                         name="Antall saker", marker_color=box_color), 
                  secondary_y=False)
    
    fig.add_trace(go.Scatter(x=df[_automatisk]["År-måned"], y=df[_automatisk]["Andel"], 
                             name="Automatisk", mode="lines+markers",
                             line={'dash': 'solid', 'color': 'black'}),
                  
                  secondary_y=True)
    fig.add_trace(go.Scatter(x=df[~_automatisk & ~_manuell]["År-måned"], y=df[~_automatisk & ~_manuell]["Andel"], 
                             name="Del-automatisk", mode="lines",
                             line={'dash': 'solid', 'color': 'grey'}), 
                  secondary_y=True)
    
    fig.update_yaxes(title_text="Andel automatisert", secondary_y=True, range=[0,1], tickformat='.0%')
    fig.update_yaxes(title_text="Antall saker", range=[0,max(df_tot["Antall innkomne saker"])*1.1], secondary_y=False)
    fig.layout.yaxis2.showgrid = False
    fig.update_layout(
        autosize=False,
        width=1300,
        height=500,)
    return fig

In [ ]:
def plot_automatisering2(df):
    '''
    df: en verdi per måned, automatiseringsgrad
    fig: Stacked box for automatiseringsgrad, line for totalt antall
    '''
    df_tot = df.groupby("År-måned", as_index=False)["Antall innkomne saker"].sum()
    
    _automatisk = df["Automatiseringsgrad"] == "Automatisk"
    _manuell = df["Automatiseringsgrad"] == "Manuell"
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    fig.add_trace(go.Scatter(x=df_tot["År-måned"], y=df_tot["Antall innkomne saker"], mode="lines", marker_color="rgb(0,0,0)", name="Totalt antall saker"), secondary_y=True)
    
    fig.add_trace(go.Bar(x=df[_automatisk]["År-måned"], y=df[_automatisk]["Andel"], marker_color=automatiserings_colors[0], name="Automatisk"), secondary_y=False)
    fig.add_trace(go.Bar(x=df[~_automatisk & ~_manuell]["År-måned"], y=df[~_automatisk & ~_manuell]["Andel"], marker_color=automatiserings_colors[1], name="Del-automatisk"), secondary_y=False)
    fig.add_trace(go.Bar(x=df[_manuell]["År-måned"], y=df[_manuell]["Andel"], marker_color=automatiserings_colors[2], name="Manuell"), secondary_y=False)
    
    fig.update_layout(yaxis_tickformat = '.0%', barmode='stack')
    fig.update_yaxes(title_text="Andel", secondary_y=False)
    fig.update_yaxes(title_text="Antall saker", range=[0,max(df_tot["Antall innkomne saker"]*1.1)], secondary_y=True)
    
    return fig

In [ ]:
def plot_automatisering_subplots(df_list, titles, box_color=px.colors.qualitative.Pastel[0]):
    '''
    df: en verdi per måned og automatiseringsgrad
    box_color: Farge på boxplot
    fig: Box for antall saker, line for automatiseringsgrad
    '''
    N = len(df_list)
    M = len(df_list[0])
    fig = make_subplots(rows=N, cols=M, specs=[[{"secondary_y": True} for i in range(N*M)]], subplot_titles=titles)
    showlegend = True
    for i in range(N):
        for j in range(M):
            if i+j > 0:
                showlegend = False
            df = df_list[i][j]
            df_tot = df.groupby("År-måned", as_index=False)["Antall innkomne saker"].sum()

            _automatisk = df["Automatiseringsgrad"] == "Automatisk"
            _manuell = df["Automatiseringsgrad"] == "Manuell"
    
            fig.add_trace(go.Bar(x=df_tot["År-måned"], y=df_tot["Antall innkomne saker"], 
                                 name="Antall saker", legendgroup="Antall saker", showlegend=showlegend, marker_color=box_color),
                          row=i+1, col=j+1, secondary_y=False)

            fig.add_trace(go.Scatter(x=df[_automatisk]["År-måned"], y=df[_automatisk]["Andel"], 
                                     name="Automatisk", legendgroup="Automatisk", showlegend=showlegend, mode="lines+markers",
                                     line={'dash': 'solid', 'color': 'black'}),
                          row=i+1, col=j+1, secondary_y=True)
            
            fig.add_trace(go.Scatter(x=df[~_automatisk & ~_manuell]["År-måned"], y=df[~_automatisk & ~_manuell]["Andel"], 
                                     name="Del-automatisk", legendgroup="Del-automatisk", showlegend=showlegend, mode="lines",
                                     line={'dash': 'solid', 'color': 'grey'}), 
                          row=i+1, col=j+1, secondary_y=True)
    
            fig.update_yaxes(title_text="Andel automatisert", secondary_y=True, range=[0,1], tickformat='.0%', row=i+1, col=j+1)
            fig.update_yaxes(title_text="Antall saker", range=[0,max(df_tot["Antall innkomne saker"])*1.1], secondary_y=False, row=i+1, col=j+1)
    
    fig.layout.yaxis2.showgrid = False
    fig.update_layout(
        autosize=False,
        width=1300,
        height=500,)
    return fig

## Total automatiseringsgrad

In [ ]:
df_auto = df_ap.groupby(["År-måned", "Automatiseringsgrad"], as_index=False)["Antall innkomne saker"].sum()
df_auto = add_total_and_andel(df_auto)

In [ ]:
fig_automatisering_total1 = plot_automatisering1(df_auto)
fig_automatisering_total1.show()

## Automatisering på selvbetjente saker

In [ ]:
df_selvbetjent = df_ap[_selvbetjent].groupby(["År-måned", "Automatiseringsgrad"], as_index=False)[["Antall innkomne saker"]].sum()
df_selvbetjent = add_total_and_andel(df_selvbetjent)

#fig_selvbetjent = px.bar(df_selvbetjent, x="År-måned", y="Andel", color="Automatiseringsgrad", color_discrete_sequence=automatiserings_colors)
#fig_selvbetjent.update_layout(yaxis_tickformat = '.0%')
#fig_selvbetjent.show()

In [ ]:
fig_automatisering_selvbetjent1 = plot_automatisering1(df_selvbetjent)
fig_automatisering_selvbetjent1.show()

## Automatisering på ikke selvbetjente saker

In [ ]:
df_ikke_selvbetjent = df_ap[~_selvbetjent].groupby(["År-måned", "Automatiseringsgrad"], as_index=False)[["Antall innkomne saker"]].sum()
df_ikke_selvbetjent = add_total_and_andel(df_ikke_selvbetjent)

In [ ]:
fig_automatisering_ikke_selvbetjent1 = plot_automatisering1(df_ikke_selvbetjent)
fig_automatisering_ikke_selvbetjent1.show()

In [ ]:
titles = ('Selvbetjent', 'Ikke selvbetjent')
fig_automatisering_selvbetjening = plot_automatisering_subplots([[df_selvbetjent, df_ikke_selvbetjent]], titles)
fig_automatisering_selvbetjening.show()

## Automatiseringsgrad Utlandstilsnitt

In [ ]:
df_automatisering_innland = df_ap[_innland].groupby(["År-måned", "Automatiseringsgrad"], as_index=False)["Antall innkomne saker"].sum()
df_automatisering_innland = add_total_and_andel(df_automatisering_innland)

In [ ]:
df_automatisering_utland = df_ap[~_innland].groupby(["År-måned", "Automatiseringsgrad"], as_index=False)["Antall innkomne saker"].sum()
df_automatisering_utland = add_total_and_andel(df_automatisering_utland)

In [ ]:
fig_automatisering_utlandstilsnitt = plot_automatisering_subplots([[df_automatisering_innland, df_automatisering_utland]], titles=["Innland", "Utland"])
fig_automatisering_utlandstilsnitt.show()

# Selvbetjeningsgrad

In [ ]:
def plot_selvbetjening1(df, box_color=px.colors.qualitative.Pastel[2]):

    df_tot = df.groupby("År-måned", as_index=False)["Antall innkomne saker"].sum()
    
    _selvbetjent = df["Selvbetjening"] == "Selvbetjent"
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    fig.add_trace(go.Bar(x=df_tot["År-måned"], y=df_tot["Antall innkomne saker"], 
                         name="Antall saker", marker_color=box_color), 
                  secondary_y=False)
    
    fig.add_trace(go.Scatter(x=df[_selvbetjent]["År-måned"], y=df[_selvbetjent]["Andel"], 
                             name="Selvbetjening", mode="lines+markers",
                             line={'dash': 'solid', 'color': 'black'}),
                  
                  secondary_y=True)
    
    fig.update_yaxes(title_text="Andel selvbetjening", secondary_y=True, range=[0,1], tickformat='.0%')
    fig.update_yaxes(title_text="Antall saker", range=[0,max(df_tot["Antall innkomne saker"])*1.1], secondary_y=False)
    fig.layout.yaxis2.showgrid = False
    fig.update_layout(
        autosize=False,
        width=1300,
        height=500,)    
    return fig

In [ ]:
def plot_selvbetjening2(df):
    
    df_tot = df.groupby("År-måned", as_index=False)["Antall innkomne saker"].sum()
    
    _selvbetjent = df["Selvbetjening"] == "Selvbetjent"
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    fig.add_trace(go.Scatter(x=df_tot["År-måned"], y=df_tot["Antall innkomne saker"], mode="lines", marker_color="rgb(0,0,0)", name="Totalt antall saker"), secondary_y=True)
    
    fig.add_trace(go.Bar(x=df[_selvbetjent]["År-måned"], y=df[_selvbetjent]["Andel"], marker_color=automatiserings_colors[0], name="Selvbetjent"), secondary_y=False)
    fig.add_trace(go.Bar(x=df[~_selvbetjent]["År-måned"], y=df[~_selvbetjent]["Andel"], marker_color=automatiserings_colors[2], name="Ikke selvbetjent"), secondary_y=False)
    
    fig.update_layout(yaxis_tickformat = '.0%', barmode='stack')
    fig.update_yaxes(title_text="Andel selvbetjening", secondary_y=False)
    fig.update_yaxes(title_text="Antall saker", range=[0,max(df_tot["Antall innkomne saker"])*1.1], secondary_y=True)
    
    return fig

In [ ]:
def plot_selvbetjening_subplots(df_list, titles, box_color=px.colors.qualitative.Pastel[2]):
    '''
    '''
    N = len(df_list)
    M = len(df_list[0])
    fig = make_subplots(rows=N, cols=M, specs=[[{"secondary_y": True} for i in range(N*M)]], subplot_titles=titles)
    showlegend = True
    for i in range(N):
        for j in range(M):
            if i+j > 0:
                showlegend = False
            df = df_list[i][j]
            df_tot = df.groupby("År-måned", as_index=False)["Antall innkomne saker"].sum()

            _selvbetjent = df["Selvbetjening"] == "Selvbetjent"
            
            fig.add_trace(go.Bar(x=df_tot["År-måned"], y=df_tot["Antall innkomne saker"], 
                         name="Antall saker", legendgroup="Antall saker", showlegend=showlegend, marker_color=box_color), 
                  row=i+1, col=j+1, secondary_y=False)
    
            fig.add_trace(go.Scatter(x=df[_selvbetjent]["År-måned"], y=df[_selvbetjent]["Andel"], 
                             name="Selvbetjening", legendgroup="Selvbetjening", showlegend=showlegend, mode="lines+markers",
                             line={'dash': 'solid', 'color': 'black'}),
                  row=i+1, col=j+1, secondary_y=True)
            
            fig.update_yaxes(title_text="Andel selvbetjening", secondary_y=True, range=[0,1], tickformat='.0%', row=i+1, col=j+1)
            fig.update_yaxes(title_text="Antall saker", range=[0,max(df_tot["Antall innkomne saker"])*1.1], secondary_y=False, row=i+1, col=j+1)
    
    fig.layout.yaxis2.showgrid = False
    fig.update_layout(
        autosize=False,
        width=1300,
        height=500,)
    return fig

## Total selvbetjeningsgrad

In [ ]:
df_andel_selvbetjening = df_ap.groupby(["År-måned", "Selvbetjening"], as_index=False)["Antall innkomne saker"].sum()
df_andel_selvbetjening = add_total_and_andel(df_andel_selvbetjening)

In [ ]:
fig_selvbetjening_total1 = plot_selvbetjening1(df_andel_selvbetjening)
fig_selvbetjening_total1.show()

## Selvbetjening innland

In [ ]:
df_selvbetjening_innland = df_ap[_innland].groupby(["År-måned", "Selvbetjening"], as_index=False)["Antall innkomne saker"].sum()
df_selvbetjening_innland = add_total_and_andel(df_selvbetjening_innland)

## Selvbetjening utland

In [ ]:
df_selvbetjening_utland = df_ap[~_innland].groupby(["År-måned", "Selvbetjening"], as_index=False)["Antall innkomne saker"].sum()
df_selvbetjening_utland = add_total_and_andel(df_selvbetjening_utland)

In [ ]:
fig_selvbetjening_utlandstilsnitt = plot_selvbetjening_subplots([[df_selvbetjening_innland, df_selvbetjening_utland]], ["Innland", "Utland"])
fig_selvbetjening_utlandstilsnitt.show()

In [ ]:
innledning = f"Statistikk på automatiserings- og selvbetjeningsgrad for førstegangsbehandling av {ytelse}. Data er basert på rapporten PSAK120. Saker opprettet av batch er eksludert."

In [ ]:
from datastory import DataStory

In [ ]:
ds = DataStory(f"Automatiserings- og Selvbetjeningsgrad for {ytelse}")

ds.markdown(innledning)

ds.header("Automatiseringsgrad", level=1)
ds.markdown("Automatiseringsgrad på alle nye saker. Del-automatisk henviser til saker hvor deler av saksbehandlingen ble gjort automatisk.")
ds.plotly(fig_automatisering_total1.to_json())

ds.header("Automatiseringsgrad selvbetjening", level=2)
ds.plotly(fig_automatisering_selvbetjening.to_json())

ds.header("Automatiseringsgrad utlandstilsnitt", level=2)
ds.plotly(fig_automatisering_utlandstilsnitt.to_json())

ds.header("Selvbetjeningsgrad", level=1)
ds.plotly(fig_selvbetjening_total1.to_json())

ds.header("Selvbetjeningsgrad utlandstilsnitt", level=2)
ds.plotly(fig_selvbetjening_utlandstilsnitt.to_json())

In [ ]:
ds.publish(url="https://nada.ekstern.dev.nav.no/api") #dev
#ds.publish(url="https://nada.intern.nav.no/api") #prod
#ds.update(url="https://nada.ekstern.dev.nav.no/api", token='1f79f29a-b9a6-48d4-8514-96a2c45ef319')